In [3]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [5]:
# read in with pandas
winter1 = pd.read_csv ('elm20-21data.txt')
winter2 = pd.read_csv ('elm21-22data.txt')
winter3 = pd.read_csv ('elm22-23data.txt')

In [7]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [9]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [11]:
precip_days2

,station,datetime,precip(mm),wxcodes
3,ELM,2021-12-01 01:53:00,T,-SN
5,ELM,2021-12-01 03:44:00,T,-RA
29,ELM,2021-12-02 02:53:00,T,-SN
30,ELM,2021-12-02 03:53:00,T,-RA
31,ELM,2021-12-02 04:53:00,0.25,-RA
...,...,...,...,...
2796,ELM,2022-02-25 11:12:00,1.02,UP BR
2797,ELM,2022-02-25 11:37:00,1.02,-SN BR
2798,ELM,2022-02-25 11:53:00,1.27,UP BR
2799,ELM,2022-02-25 12:03:00,T,UP BR


In [13]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [15]:
precip_days3[660:700]

,station,datetime,precip(mm),wxcodes
2469,ELM,2023-02-11 01:53:00,0.0001,-SN
2470,ELM,2023-02-11 02:18:00,0.0001,-SN
2472,ELM,2023-02-11 03:53:00,0.0001,-SN
2633,ELM,2023-02-17 10:53:00,0.25,-RA
2636,ELM,2023-02-17 12:11:00,0.76,RA BR
2637,ELM,2023-02-17 12:16:00,0.76,-RA BR
2639,ELM,2023-02-17 12:53:00,0.76,-RA
2640,ELM,2023-02-17 13:53:00,0.0001,-RA
2641,ELM,2023-02-17 14:20:00,0.0001,-RA
2642,ELM,2023-02-17 14:46:00,0.0001,-RA


In [17]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [20]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 53]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 53]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 53]

In [22]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [11]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [6]:
elm = LocationInfo(
    'Elmira', 'New York', 
    latitude= 42.0898,
    longitude=  -76.8077, 
    timezone='America/New_York'
)

elm

NameError: name 'LocationInfo' is not defined

In [26]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(elm.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [28]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(elm.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [30]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(elm.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [32]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:16:46  2020-12-01 16:36:01  2020-12-01
1   2020-12-02 07:17:48  2020-12-02 16:35:46  2020-12-02
2   2020-12-03 07:18:49  2020-12-03 16:35:33  2020-12-03
3   2020-12-04 07:19:49  2020-12-04 16:35:23  2020-12-04
4   2020-12-05 07:20:48  2020-12-05 16:35:15  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:49:58  2021-02-24 17:51:22  2021-02-24
86  2021-02-25 06:48:26  2021-02-25 17:52:35  2021-02-25
87  2021-02-26 06:46:53  2021-02-26 17:53:48  2021-02-26
88  2021-02-27 06:45:19  2021-02-27 17:55:01  2021-02-27
89  2021-02-28 06:43:44  2021-02-28 17:56:14  2021-02-28

[90 rows x 3 columns]


In [34]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:16:30  2021-12-01 16:36:04  2021-12-01
1   2021-12-02 07:17:33  2021-12-02 16:35:49  2021-12-02
2   2021-12-03 07:18:34  2021-12-03 16:35:35  2021-12-03
3   2021-12-04 07:19:35  2021-12-04 16:35:25  2021-12-04
4   2021-12-05 07:20:34  2021-12-05 16:35:16  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:50:21  2022-02-24 17:51:04  2022-02-24
86  2022-02-25 06:48:48  2022-02-25 17:52:17  2022-02-25
87  2022-02-26 06:47:15  2022-02-26 17:53:30  2022-02-26
88  2022-02-27 06:45:41  2022-02-27 17:54:43  2022-02-27
89  2022-02-28 06:44:07  2022-02-28 17:55:56  2022-02-28

[90 rows x 3 columns]


In [36]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 07:16:15  2022-12-01 16:36:08  2022-12-01
1   2022-12-02 07:17:18  2022-12-02 16:35:52  2022-12-02
2   2022-12-03 07:18:20  2022-12-03 16:35:38  2022-12-03
3   2022-12-04 07:19:20  2022-12-04 16:35:27  2022-12-04
4   2022-12-05 07:20:19  2022-12-05 16:35:18  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:50:43  2023-02-24 17:50:46  2023-02-24
86  2023-02-25 06:49:11  2023-02-25 17:51:59  2023-02-25
87  2023-02-26 06:47:38  2023-02-26 17:53:13  2023-02-26
88  2023-02-27 06:46:04  2023-02-27 17:54:26  2023-02-27
89  2023-02-28 06:44:30  2023-02-28 17:55:38  2023-02-28

[90 rows x 3 columns]


In [38]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [40]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ELM,2020-12-01 00:53:00,0.0001,-RA BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
1,ELM,2020-12-01 08:53:00,0.2500,-RA BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
2,ELM,2020-12-01 09:53:00,0.2500,-RA BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
3,ELM,2020-12-01 10:53:00,0.7600,-RA,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
4,ELM,2020-12-01 11:53:00,0.0001,-RA,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
...,...,...,...,...,...,...,...
364,ELM,2021-02-23 15:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08
365,ELM,2021-02-23 20:53:00,0.0001,-RA,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08
366,ELM,2021-02-24 20:53:00,0.0001,-RA,2021-02-24,2021-02-24 06:49:58,2021-02-24 17:51:22
367,ELM,2021-02-24 21:53:00,0.0001,-RA,2021-02-24,2021-02-24 06:49:58,2021-02-24 17:51:22


In [42]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ELM,2021-12-01 01:53:00,0.0001,-SN,2021-12-01,2021-12-01 07:16:30,2021-12-01 16:36:04
1,ELM,2021-12-02 02:53:00,0.0001,-SN,2021-12-02,2021-12-02 07:17:33,2021-12-02 16:35:49
2,ELM,2021-12-02 03:53:00,0.0001,-RA,2021-12-02,2021-12-02 07:17:33,2021-12-02 16:35:49
3,ELM,2021-12-02 04:53:00,0.2500,-RA,2021-12-02,2021-12-02 07:17:33,2021-12-02 16:35:49
4,ELM,2021-12-02 05:53:00,0.7600,-RA,2021-12-02,2021-12-02 07:17:33,2021-12-02 16:35:49
...,...,...,...,...,...,...,...
252,ELM,2022-02-25 08:53:00,3.0500,UP BR,2022-02-25,2022-02-25 06:48:48,2022-02-25 17:52:17
253,ELM,2022-02-25 09:53:00,2.2900,UP BR,2022-02-25,2022-02-25 06:48:48,2022-02-25 17:52:17
254,ELM,2022-02-25 10:53:00,1.7800,UP BR,2022-02-25,2022-02-25 06:48:48,2022-02-25 17:52:17
255,ELM,2022-02-25 11:53:00,1.2700,UP BR,2022-02-25,2022-02-25 06:48:48,2022-02-25 17:52:17


In [44]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ELM,2022-12-03 09:53:00,0.0001,-RA,2022-12-03,2022-12-03 07:18:20,2022-12-03 16:35:38
1,ELM,2022-12-03 10:53:00,0.5100,RA,2022-12-03,2022-12-03 07:18:20,2022-12-03 16:35:38
2,ELM,2022-12-03 11:53:00,1.2700,-RA,2022-12-03,2022-12-03 07:18:20,2022-12-03 16:35:38
3,ELM,2022-12-03 15:53:00,0.0001,-RA,2022-12-03,2022-12-03 07:18:20,2022-12-03 16:35:38
4,ELM,2022-12-03 17:53:00,0.0001,-RA,2022-12-03,2022-12-03 07:18:20,2022-12-03 16:35:38
...,...,...,...,...,...,...,...
282,ELM,2023-02-25 16:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:49:11,2023-02-25 17:51:59
283,ELM,2023-02-25 17:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:49:11,2023-02-25 17:51:59
284,ELM,2023-02-25 23:53:00,0.0001,-SN,2023-02-25,2023-02-25 06:49:11,2023-02-25 17:51:59
285,ELM,2023-02-27 22:53:00,1.0200,-SN BR,2023-02-27,2023-02-27 06:46:04,2023-02-27 17:54:26


In [46]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
1,ELM,2020-12-01 08:53:00,0.2500,-RA BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
2,ELM,2020-12-01 09:53:00,0.2500,-RA BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
3,ELM,2020-12-01 10:53:00,0.7600,-RA,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
4,ELM,2020-12-01 11:53:00,0.0001,-RA,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
5,ELM,2020-12-01 12:53:00,0.2500,-RA,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
...,...,...,...,...,...,...,...
360,ELM,2021-02-22 15:53:00,0.5100,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54
361,ELM,2021-02-22 16:53:00,0.0001,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54
363,ELM,2021-02-23 14:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08
364,ELM,2021-02-23 15:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08


In [48]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [50]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
1,ELM,2020-12-01 08:53:00,0.2500,-RA BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01,2020-12-01 08:16:46,2020-12-01 15:36:01
2,ELM,2020-12-01 09:53:00,0.2500,-RA BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01,2020-12-01 08:16:46,2020-12-01 15:36:01
3,ELM,2020-12-01 10:53:00,0.7600,-RA,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01,2020-12-01 08:16:46,2020-12-01 15:36:01
4,ELM,2020-12-01 11:53:00,0.0001,-RA,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01,2020-12-01 08:16:46,2020-12-01 15:36:01
5,ELM,2020-12-01 12:53:00,0.2500,-RA,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01,2020-12-01 08:16:46,2020-12-01 15:36:01
...,...,...,...,...,...,...,...,...,...
359,ELM,2021-02-22 14:53:00,0.7600,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54,2021-02-22 07:53:01,2021-02-22 16:48:54
360,ELM,2021-02-22 15:53:00,0.5100,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54,2021-02-22 07:53:01,2021-02-22 16:48:54
363,ELM,2021-02-23 14:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08,2021-02-23 07:51:30,2021-02-23 16:50:08
364,ELM,2021-02-23 15:53:00,0.0001,-SN,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08,2021-02-23 07:51:30,2021-02-23 16:50:08


In [52]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [54]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [56]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 0.21334214285714284
2021-2022: 0.4466855421686748
2022-2023: 0.4462038461538461


In [58]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,ELM,2020-12-01 00:53:00,0.0001,-RA BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
8,ELM,2020-12-01 16:53:00,0.0001,-RA,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
9,ELM,2020-12-01 17:53:00,0.5100,-SN BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
10,ELM,2020-12-01 22:53:00,0.2500,-SN BR,2020-12-01,2020-12-01 07:16:46,2020-12-01 16:36:01
11,ELM,2020-12-02 00:53:00,0.0001,-SN,2020-12-02,2020-12-02 07:17:48,2020-12-02 16:35:46
...,...,...,...,...,...,...,...
355,ELM,2021-02-21 06:53:00,0.0001,-SN,2021-02-21,2021-02-21 06:54:30,2021-02-21 17:47:39
362,ELM,2021-02-22 22:53:00,0.0001,-SN,2021-02-22,2021-02-22 06:53:01,2021-02-22 17:48:54
365,ELM,2021-02-23 20:53:00,0.0001,-RA,2021-02-23,2021-02-23 06:51:30,2021-02-23 17:50:08
366,ELM,2021-02-24 20:53:00,0.0001,-RA,2021-02-24,2021-02-24 06:49:58,2021-02-24 17:51:22


In [60]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_26432\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [62]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [64]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [66]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.49524197530864195
2021-2022: 0.31360789473684214
2022-2023: 0.45386842105263164


In [68]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('elm.csv', index=False)